### 欠損値の特定

In [317]:
import numpy
import pandas
from IPython.core.display import display

# 欠損値を含むデータを作成
data = {'name': ['Ryo', 'Kaori', 'Hideyuki', 'Hayato', 'Miki', 'Saeko'],  # 名前
        'gender': ['M', 'F', 'M', 'M', 'F', 'F'],  # 性別
        'height': [186, 168, 175, 210, 160, 163],  # 身長
        'weight': [72, 47, 62, 90, None, numpy.NaN],  # 体重
        'age': [30, 20, None , numpy.NaN, 23, 25],  # 年齢
        'size': ['L', 'M', 'L', 'XL', None, 'S']  # 服のサイズ
        }
columns = ['name', 'gender', 'age', 'height', 'weight', 'size']
# DataFrameを作成
df = pandas.DataFrame(data, columns=columns)
display(df)
display(df.dtypes)
display(df.info)

,name,gender,age,height,weight,size
0,Ryo,M,30.0,186,72.0,L
1,Kaori,F,20.0,168,47.0,M
2,Hideyuki,M,NaN,175,62.0,L
3,Hayato,M,NaN,210,90.0,XL
4,Miki,F,23.0,160,NaN,None
5,Saeko,F,25.0,163,NaN,S


name       object
gender     object
age       float64
height      int64
weight    float64
size       object
dtype: object

<bound method DataFrame.info of        name gender   age  height  weight  size
0       Ryo      M  30.0     186    72.0     L
1     Kaori      F  20.0     168    47.0     M
2  Hideyuki      M   NaN     175    62.0     L
3    Hayato      M   NaN     210    90.0    XL
4      Miki      F  23.0     160     NaN  None
5     Saeko      F  25.0     163     NaN     S>

In [318]:
# 欠損値データを確認
df.isnull()

,name,gender,age,height,weight,size
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,True,False,False,False
3,False,False,True,False,False,False
4,False,False,False,False,True,True
5,False,False,False,False,True,False


In [319]:
# 欠損値データの合計数を確認
df.isnull().sum()

name      0
gender    0
age       2
height    0
weight    2
size      1
dtype: int64

### 欠損値の削除

#### ・欠損値を含む行を削除する

In [320]:
display(df.dropna())
'''
欠損値を含む行を削除したいときは、dropna()メソッドを呼ぶ
デフォルトはdropna(axis=0)
'''

,name,gender,age,height,weight,size
0,Ryo,M,30.0,186,72.0,L
1,Kaori,F,20.0,168,47.0,M


'\n欠損値を含む行を削除したいときは、dropna()メソッドを呼ぶ\nデフォルトはdropna(axis=0)\n'

#### ・欠損値を含む列を削除する

In [321]:
display(df.dropna(axis=1))

,name,gender,height
0,Ryo,M,186
1,Kaori,F,168
2,Hideyuki,M,175
3,Hayato,M,210
4,Miki,F,160
5,Saeko,F,163


### 欠損値の補間

#### ・数字データの補間

In [322]:
# 数字データの補間ライブラリのImputerの読み込み
from sklearn.preprocessing import Imputer

In [323]:
# Imputerのインスタンスを作成
imp_num = Imputer(missing_values='NaN', strategy='mean', axis=0)
'''
missing_valuesに欠損値と見なす値
strategyに補間方法（ "mean","median","most_frequent"）を与える。
'''

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


'\nmissing_valuesに欠損値と見なす値\nstrategyに補間方法（ "mean","median","most_frequent"）を与える。\n'

### 欠損値補間の自動化

In [324]:
def impute(df, strategy='mean'):
    from sklearn.preprocessing import Imputer
    imp_num = Imputer(missing_values='NaN', strategy=strategy, axis=0)
    # 欠損がある列を取り出す。
    isnull_sums = df.isnull().sum() #欠損値の数を計算
    isnull_index = []
    for index, isnull_sum in enumerate(isnull_sums): #欠損値の数が0より多い場合のインデックスを抽出
        if isnull_sum > 0:
            isnull_index.append(index)
            isnull_df = df.iloc[:, isnull_index]

    display('欠損値があるデータの抽出',isnull_df)
    print('-------------------------------')
    display('データ型の確認',isnull_df.dtypes)
    print('-------------------------------')

    # 数字データのみの列を取り出す
    df_nums = df.iloc[:, isnull_index].select_dtypes(include=[int, float])
    df_not_nums = df.iloc[:, isnull_index].select_dtypes(exclude=[int, float])

    if not df_nums.empty:
        display('以下のデータを補間します。',df_nums)
        print('-------------------------------')

        # インスタンス化した imp_num に与えて、欠損値を補間
        imputed_data = imp_num.fit_transform(df_nums)
        print(imputed_data)
        df_nums[:] = imputed_data
        display('以下のデータで補間します。',df_nums)
        print('-------------------------------')

        # 元のdfに代入
        df[df_nums.columns] = df_nums
    display('補間完了',df)
    print('-------------------------------')

    if not df_not_nums.empty:
        display('以下は数値データでは無いので欠損値の補間が出来ません。',df_not_nums)


In [325]:
# 自動化の確認
impute(df)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


'欠損値があるデータの抽出'

,age,weight,size
0,30.0,72.0,L
1,20.0,47.0,M
2,NaN,62.0,L
3,NaN,90.0,XL
4,23.0,NaN,None
5,25.0,NaN,S


-------------------------------


'データ型の確認'

...

'補間完了'

,name,gender,age,height,weight,size
0,Ryo,M,30.0,186,72.00,L
1,Kaori,F,20.0,168,47.00,M
2,Hideyuki,M,24.5,175,62.00,L
3,Hayato,M,24.5,210,90.00,XL
4,Miki,F,23.0,160,67.75,None
5,Saeko,F,25.0,163,67.75,S


-------------------------------


'以下は数値データでは無いので欠損値の補間が出来ません。'

,size
0,L
1,M
2,L
3,XL
4,None
5,S


### カテゴリカルデータの補間

In [326]:
# size2int:size to int
size2int = {'S': 1, 'M': 2, 'L': 3, 'XL': 4}
data_column = 'size'
# int2size:int to size
# size2intの逆をfor文で入力する
int2size = {v:k for k, v in size2int.items()}

# カテゴリカル・データ補間の自動化
def categorical(data, size2int, data_column):
    data[data_column] = data[data_column].map(size2int)
    display(data)

# 自動化確認
categorical(df, size2int, data_column)
# 最頻値で補間を行う
impute(df, 'most_frequent')

,name,gender,age,height,weight,size
0,Ryo,M,30.0,186,72.00,3.0
1,Kaori,F,20.0,168,47.00,2.0
2,Hideyuki,M,24.5,175,62.00,3.0
3,Hayato,M,24.5,210,90.00,4.0
4,Miki,F,23.0,160,67.75,NaN
5,Saeko,F,25.0,163,67.75,1.0


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


'欠損値があるデータの抽出'

,size
0,3.0
1,2.0
2,3.0
3,4.0
4,NaN
5,1.0


-------------------------------


...

,size
0,3.0
1,2.0
2,3.0
3,4.0
4,3.0
5,1.0


-------------------------------


'補間完了'

,name,gender,age,height,weight,size
0,Ryo,M,30.0,186,72.00,3.0
1,Kaori,F,20.0,168,47.00,2.0
2,Hideyuki,M,24.5,175,62.00,3.0
3,Hayato,M,24.5,210,90.00,4.0
4,Miki,F,23.0,160,67.75,3.0
5,Saeko,F,25.0,163,67.75,1.0


-------------------------------


### カテゴリカル・データのダミー化

In [327]:
import numpy
import pandas
from IPython.core.display import display

# 欠損値を含むデータを作成
data = {'name': ['Ryo', 'Kaori', 'Hideyuki', 'Hayato', 'Miki', 'Saeko'],  # 名前
        'gender': ['M', 'F', 'M', 'M', 'F', 'F'],  # 性別
        'height': [186, 168, 175, 210, 160, 163],  # 身長
        'weight': [72, 47, 62, 90, None, numpy.NaN],  # 体重
        'age': [30, 20, None , numpy.NaN, 23, 25],  # 年齢
        'size': ['L', 'M', 'L', 'XL', None, 'S']  # 服のサイズ
        }
columns = ['name', 'gender', 'age', 'height', 'weight', 'size']
# DataFrameを作成
df = pandas.DataFrame(data, columns=columns)
display(df)

,name,gender,age,height,weight,size
0,Ryo,M,30.0,186,72.0,L
1,Kaori,F,20.0,168,47.0,M
2,Hideyuki,M,NaN,175,62.0,L
3,Hayato,M,NaN,210,90.0,XL
4,Miki,F,23.0,160,NaN,None
5,Saeko,F,25.0,163,NaN,S


In [328]:
# ダミー化の自動化
def dummy(data, data_column, categories):
    # size の列のデータタイプを category に変換
    data[data_column] = pandas.Categorical(data[data_column], categories=categories, ordered=True)

    # get_dummies()関数を利用してダミー化,dummy_na=Trueの引数を与える。
    data = pandas.get_dummies(data, columns=[data_column], dummy_na=True)

    display(data)
    
# 自動化の確認
dummy(df, 'size', ['S', 'M', 'L', 'XL'])

,name,gender,age,height,weight,size_S,size_M,size_L,size_XL,size_nan
0,Ryo,M,30.0,186,72.0,0,0,1,0,0
1,Kaori,F,20.0,168,47.0,0,1,0,0,0
2,Hideyuki,M,NaN,175,62.0,0,0,1,0,0
3,Hayato,M,NaN,210,90.0,0,0,0,1,0
4,Miki,F,23.0,160,NaN,0,0,0,0,1
5,Saeko,F,25.0,163,NaN,1,0,0,0,0
